Q1. Getting the embeddings model

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
embedding_model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(embedding_model_name)

In [5]:
user_question = "I just discovered the course. Can I still join it?"
user_question_embedding = embedding_model.encode(user_question)

In [6]:
user_question_embedding[0]

0.078222655

In [7]:
import requests

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [9]:
course_name = "machine-learning-zoomcamp"
filtered_documents = [item for item in documents if item["course"] == course_name]

In [10]:
len(filtered_documents)

375

Q2. Creating the embeddings

In [11]:
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(filtered_documents):
    qa_text = f"{doc['question']} {doc['text']}"
    doc_embeddings = embedding_model.encode(qa_text)
    embeddings.append(doc_embeddings)

embeddings[0]

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [00:48<00:00,  7.66it/s]


array([ 8.80591124e-02,  1.55936033e-02,  7.92557523e-02,  2.52757687e-02,
        7.55764917e-02, -3.90596800e-02, -4.13814038e-02,  2.52916943e-02,
        2.43241936e-02,  3.62587790e-03, -7.28286849e-03, -3.28751504e-02,
        6.12956285e-02, -5.71100414e-02,  1.16774449e-02, -1.79441702e-02,
        4.49206121e-02, -5.41606732e-02, -1.92255690e-03,  1.48329614e-02,
        7.91351125e-03, -3.43127102e-02,  1.21481167e-02,  1.27186347e-02,
       -1.60092004e-02,  7.12132733e-03,  1.58389080e-02, -3.27576447e-04,
        3.01294727e-03, -4.58917543e-02, -1.34771131e-03,  2.01150123e-02,
        2.33724173e-02, -4.28203307e-03, -3.09383031e-02,  1.94304925e-03,
       -1.88219491e-02,  5.35158208e-03,  1.29352380e-02, -3.23958807e-02,
        3.10556982e-02,  5.12981601e-03, -3.24515800e-04, -3.15542109e-02,
        1.93377640e-02, -6.38601482e-02, -3.05381212e-02, -6.27885312e-02,
       -1.74859818e-02,  3.52948979e-02, -2.98071187e-02,  7.45345950e-02,
       -2.06182245e-02, -

In [12]:
import numpy as np

In [25]:
X = np.array(embeddings)

# ↓(rows) →(columns)
X.shape

(375, 768)

In [21]:
X[374]

array([ 4.38601151e-02, -1.30076660e-02,  6.37312159e-02,  3.47584337e-02,
        8.71159062e-02, -5.54476827e-02, -4.05455567e-03,  3.28896083e-02,
        1.29048759e-02, -7.41615817e-02, -2.47359760e-02, -1.64387338e-02,
        9.22459885e-02,  1.71852938e-03,  6.36736378e-02, -3.88733521e-02,
        1.55954529e-02, -3.95982340e-02, -1.61853991e-02, -3.35151292e-02,
       -3.04112397e-02, -4.30843048e-02,  2.35840436e-02,  3.48229706e-02,
       -2.31121890e-02,  3.99431176e-02,  3.21094617e-02, -4.31509130e-02,
        3.33551434e-03, -3.81750725e-02,  2.08699368e-02, -3.11478879e-02,
        6.12567440e-02,  2.68826424e-03, -1.08577292e-02, -3.77703016e-03,
        1.13106910e-02, -2.94178259e-02, -2.23544333e-02, -3.61512788e-02,
        2.93248054e-02,  1.11485384e-02,  2.36910097e-02,  3.81251089e-02,
        7.74852745e-03, -5.57733104e-02, -8.14864188e-02, -1.88939385e-02,
        4.47131544e-02,  1.93721205e-02,  9.58892703e-03,  2.70428769e-02,
        9.85949188e-02, -

Q3. Search

In [14]:
# Select a vector (for example, the first vector in X)
vector = X[0]

# The vectors returned from the embedding model are already normalized 
# (you can check it by computing a dot product of a vector with itself - it should return 1.0)
dot_product = np.dot(vector, vector)

dot_product

1.0

In [15]:
scores = X.dot(user_question_embedding)
print(scores)
print()
print(f"The answer to Q3.1: {scores.max()}")

[ 2.89217323e-01  4.35050964e-01  2.20572680e-01  1.28085867e-01
  3.28754723e-01  4.58999753e-01  4.48930800e-01  2.70170838e-01
  2.13974997e-01  3.39745611e-01  1.63339078e-01  3.15917820e-01
  1.53621100e-02  4.14240539e-01  6.50657535e-01  4.12786603e-01
  2.02390447e-01  4.71707731e-01  3.74821365e-01  3.53319347e-01
  4.50558484e-01  1.85430810e-01  3.45527023e-01  8.22948143e-02
 -2.19077673e-02 -4.64250520e-02  8.60690847e-02  9.87197980e-02
 -2.14188341e-02  1.28751695e-02  2.54869889e-02  3.66865806e-02
  4.56339307e-02 -2.05279551e-02  1.82662327e-02  3.39788646e-02
  1.66507531e-03  1.03210807e-02  3.40170003e-02 -2.65230574e-02
 -4.11910005e-02 -4.97983135e-02 -6.20346777e-02 -7.63562508e-03
  4.61617671e-02  6.95954915e-03  4.77613285e-02 -9.97651368e-04
 -5.60125411e-02 -5.96207008e-02  2.05209315e-01 -1.16968520e-01
 -2.34159678e-02 -1.52457990e-02 -6.83521200e-03 -8.61714333e-02
  2.02636980e-02 -2.51591727e-02  4.21458855e-04 -3.79212871e-02
 -3.21440250e-02  2.48612

In [ ]:
scores = embeddings.dot(query)

In [33]:
from numpy import ndarray

class VectorSearchEngine():
    def __init__(self, documents, embeddings: ndarray):
        self.documents = documents
        self.embeddings = embeddings
    
    def search(self, vector_query, result_number = 10):
        scores = self.embeddings.dot(vector_query)
        ordered = np.argsort(-scores)
        idx = ordered[:result_number]
        result = [self.documents[i] for i in idx]
        return result

search_engine = VectorSearchEngine(filtered_documents, X)
seach_result = search_engine.search(user_question_embedding, 5)

In [32]:
seach_result

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c